In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
sector = "healthcare"

In [3]:
df_analysis = pd.read_csv(sector + "_dataset.csv")
df_analysis.head()

,Date,Tick,Close,Polarity,News
0,2016-01-04,AAAP,30.67,0.0,0
1,2016-01-05,AAAP,30.88,0.0,0
2,2016-01-06,AAAP,29.96,0.0,0
3,2016-01-07,AAAP,29.65,0.0,0
4,2016-01-08,AAAP,28.00,0.0,0


<h3> Pivoting and Generating Features & Labels </h3>

In [149]:
df_analysis['Signals'] = [[x, y] for x,y in zip(df_analysis['Close'], df_analysis['Polarity'])]
df_pivot = pd.pivot_table(df_analysis, values='Signals', index='Tick', columns='Date', aggfunc=lambda x: np.array(x))
df_pivot_stock = pd.pivot_table(df_analysis, values='Close', index='Tick', columns='Date', aggfunc=lambda x: np.array(x))
df_pivot_news = pd.pivot_table(df_analysis, values='Polarity', index='Tick', columns='Date', aggfunc=lambda x: np.array(x))
df_pivot.head(), df_pivot_stock.head(), df_pivot_news.head()

(Date     2016-01-04     2016-01-05     2016-01-06  \
 Tick                                                
 AAAP   [30.67, 0.0]   [30.88, 0.0]   [29.96, 0.0]   
 ABBV   [57.61, 0.0]   [57.37, 0.0]   [57.38, 0.0]   
 ABC   [101.87, 0.0]  [103.36, 0.0]  [101.78, 0.0]   
 ABMD   [85.24, 0.0]    [85.0, 0.0]    [85.3, 0.0]   
 ACAD   [34.62, 0.0]   [34.46, 0.0]   [29.24, 0.0]   
 
 Date                   2016-01-07                    2016-01-08  \
 Tick                                                              
 AAAP                 [29.65, 0.0]                   [28.0, 0.0]   
 ABBV                 [57.21, 0.0]                  [55.65, 0.0]   
 ABC                  [98.53, 0.0]  [97.96, -0.9999990000010001]   
 ABMD                 [81.92, 0.0]                  [84.58, 0.0]   
 ACAD  [28.73, 0.9999990000010001]                  [28.04, 0.0]   
 
 Date            2016-01-09            2016-01-10                   2016-01-11  \
 Tick                                                       

<h3> Prepare input for Tensor Flow </h3>

In [190]:
window = 100
features = []
labels = []
perc_var = []

i = 0
end_col = window + 1

while end_col + i < df_pivot.shape[1]:
    features.append(np.array([df_pivot_stock.iloc[:, i:end_col+i-1].values, df_pivot_news.iloc[:, i:end_col+i-1].values]))
    l = (df_pivot.iloc[:,end_col+i].map(lambda x: x[0]) - df_pivot.iloc[:,end_col+i-1].map(lambda x: x[0])).map(lambda x: 1 if x>0 else 0).values
    labels.append(l)
    
    perc_var.append( ((df_pivot.iloc[:,end_col+i].map(lambda x: x[0]) - df_pivot.iloc[:,end_col+i-1].map(lambda x: x[0])) / df_pivot.iloc[:,end_col+i-1].map(lambda x: x[0])).values ) 
    i = i + 1

features = np.array(features)
labels = np.array(labels)
perc_var = np.array(perc_var)
features.shape, labels.shape, features[0], labels[0:2], perc_var.shape, perc_var[0]

((535, 2, 154, 100),
 (535, 154),
 array([[[  30.67      ,   30.88      ,   29.96      , ...,   35.76666667,
            35.8       ,   35.75      ],
         [  57.61      ,   57.37      ,   57.38      , ...,   58.49      ,
            58.5       ,   59.89      ],
         [ 101.87      ,  103.36      ,  101.78      , ...,   84.73      ,
            84.44      ,   84.9       ],
         ..., 
         [  58.86      ,   60.06      ,   59.19      , ...,   59.57333333,
            59.5       ,   59.71      ],
         [ 101.81      ,  103.93      ,  104.38      , ...,  109.18666667,
           109.3       ,  111.57      ],
         [  47.27      ,   48.01      ,   48.02      , ...,   46.76      ,
            46.67      ,   47.01      ]],
 
        [[   0.        ,    0.        ,    0.        , ...,    0.        ,
             0.        ,    0.        ],
         [   0.        ,    0.        ,    0.        , ...,    0.        ,
             0.        ,    0.        ],
         [   0.     

<h3> Split Train - Eval </h3>

In [174]:
train_features = features[:-100]
test_features = features[-100:]
train_labels = labels[:-100]
test_labels = labels[-100:]

train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((435, 2, 154, 100), (100, 2, 154, 100), (435, 154), (100, 154))

<h3> TRAIN CNN </h3>

In [175]:
exec(open("cnn_stocks.py").read())

In [176]:
# Create the Estimator
stocks_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="../tmp/stocks_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '../tmp/stocks_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


INFO:tensorflow:Using config: {'_model_dir': '../tmp/stocks_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [177]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_features},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

stocks_classifier.train(
    input_fn=train_input_fn,
    steps=1000,
    hooks=[logging_hook])

Tensor("Reshape:0", shape=(100, 154, 100, 2), dtype=float32)
Tensor("conv2d/Relu:0", shape=(100, 154, 100, 32), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(100, 154, 50, 32), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(100, 154, 50, 8), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(100, 154, 10, 8), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(100, 154, 10, 2), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(100, 154, 2, 2), dtype=float32)
Tensor("Reshape_1:0", shape=(100, 616), dtype=float32)
Tensor("dense/Relu:0", shape=(100, 1024), dtype=float32)
Tensor("dropout/dropout/mul:0", shape=(100, 1024), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(100, 154), dtype=float32)
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.


INFO:tensorflow:logits.dtype=<dtype: 'float32'>.


INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.


INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.


INFO:tensorflow:losses.dtype=<dtype: 'float32'>.


...

INFO:tensorflow:probabilities = [[ 0.00660419  0.00660972  0.00696774 ...,  0.00687925  0.00660178
   0.00679344]
 [ 0.0084503   0.00739729  0.0057889  ...,  0.00751557  0.00567843
   0.00680072]
 [ 0.00656769  0.0067173   0.00648431 ...,  0.00684262  0.00647919
   0.00597424]
 ..., 
 [ 0.00667643  0.00767868  0.0068862  ...,  0.00647319  0.00645448
   0.00685304]
 [ 0.00709825  0.00675711  0.00656824 ...,  0.00609003  0.00599348
   0.00789256]
 [ 0.0068777   0.00667402  0.00657188 ...,  0.00642477  0.00643214
   0.00686786]] (322.752 sec)


INFO:tensorflow:Saving checkpoints for 1000 into ../tmp/stocks_convnet_model\model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into ../tmp/stocks_convnet_model\model.ckpt.


INFO:tensorflow:Loss for final step: 0.692146.


INFO:tensorflow:Loss for final step: 0.692146.


<h3> Model Evaluation </h3>

In [258]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_features},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

eval_results = stocks_classifier.predict(input_fn=eval_input_fn)

N = 5
capital = 1
for i, r in enumerate(eval_results):
    top_n = r['probabilities'].argsort()[-N:][::-1]
    allocation = [0 for x in perc_var[i]]
    for j, t in enumerate(top_n):
        allocation[t] = capital / N
        
    variation = allocation + (allocation * perc_var[i]) 
    capital = variation.sum()
    
        
print(capital)

Tensor("Reshape:0", shape=(?, 154, 100, 2), dtype=float32)
Tensor("conv2d/Relu:0", shape=(?, 154, 100, 32), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 154, 50, 32), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 154, 50, 8), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 154, 10, 8), dtype=float32)
Tensor("conv2d_3/Relu:0", shape=(?, 154, 10, 2), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 154, 2, 2), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 616), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("dropout/Identity:0", shape=(?, 1024), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 154), dtype=float32)
INFO:tensorflow:Restoring parameters from ../tmp/stocks_convnet_model\model.ckpt-1000


INFO:tensorflow:Restoring parameters from ../tmp/stocks_convnet_model\model.ckpt-1000


1.11174054446
